In [7]:
#Подход 1. Градиентный бустинг. 
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

features = pd.read_csv('features.csv', index_col='match_id')

aim = features['radiant_win']
#Задание 1. Удаляем признаки:
features = features.drop(['radiant_win', 'duration','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'], axis = 1)
#Задание 2. Ищем признаки с пропусками:
features_with_nans = np.nonzero(features.count(axis = 0) < 97230)
print("Features with NaN:", features.axes[1][features_with_nans], len(features.axes[1][features_with_nans]))

#Рассмотрим признаки first_blood_time и radiant_first_ward_time. Оба они имеют пропуски, так как соответствующие события могли
#не случиться за первые пять минут в каких-то матчах. Например, не было "первой крови" или установки наблюдателя. Может быть, 
#наблюдатель вообще не был установлен за всю игру.

#Задание 3. Заполняем пропуски:
features = features.fillna(value = 0)
#Задание 4: Целевую переменную содержит столбец 'radiant_win'. Его мы в самом начале, перед удалением сохранили в aim
#Задание 5. Обучаем Градиентный бустинг и смотрим на качество кросс-валидации при разном количестве деревьев:
import time
import datetime

start_time = datetime.datetime.now()

grid = {'n_estimators': [10,20,30,40,50,100]} #функция AUC_ROC и predict_proba используются при gridsearch. Они вызываются внутри
cr_v = KFold(aim.size, n_folds = 5, shuffle = True, random_state = 241)
clf = GradientBoostingClassifier(random_state = 241)
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv = cr_v)
gs.fit(features, aim)
print(gs.grid_scores_)


print ('Time elapsed:', datetime.datetime.now() - start_time)
#Экспериментально видим, что с увеличением количества деревьев растет качество. То есть качество продолжит расти, отимум не достигнут.
#Лучшее качество получилось на 100 деревьях. Обучение 10,20,30,40,50 и 100 деревьев и кросс валидация заняли 24 минуты. 
#На 30 деревьях градиентный бустинг обучался около 3-х минут. Довольно долго, даже если учесть, что компьютер совсем не мощный.

Features with NaN: Index(['first_blood_time', 'first_blood_team', 'first_blood_player1',
       'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time',
       'radiant_flying_courier_time', 'radiant_first_ward_time',
       'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time',
       'dire_first_ward_time'],
      dtype='object') 12
[mean: 0.66439, std: 0.00490, params: {'n_estimators': 10}, mean: 0.68285, std: 0.00501, params: {'n_estimators': 20}, mean: 0.68950, std: 0.00453, params: {'n_estimators': 30}, mean: 0.69413, std: 0.00433, params: {'n_estimators': 40}, mean: 0.69745, std: 0.00391, params: {'n_estimators': 50}, mean: 0.70633, std: 0.00359, params: {'n_estimators': 100}]
Time elapsed: 0:24:09.175888


In [10]:
#Подход 2. Логистическая регрессия.
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

features = pd.read_csv('features.csv', index_col='match_id')

aim = features['radiant_win']

features = features.drop(['radiant_win', 'duration','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'], axis = 1)

features_with_nans = np.nonzero(features.count(axis = 0) < 97230)
print("Features with NaN:", features.axes[1][features_with_nans], len(features.axes[1][features_with_nans]))

features = features.fillna(value = 0)
#Задание 1. Отмасштабируем признаки и обучим логистическую регрессию. Оптимальный параметр получился С=0.01. 
#Мы также достигли лучшего качества за ощутимо меньшее время, чем при бустинге.
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
import time
import datetime
start_time = datetime.datetime.now()

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cr_v = KFold(aim.size, n_folds = 5, shuffle = True, random_state = 241)
clf = LogisticRegression(random_state = 241)
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv = cr_v)
gs.fit(features_scaled, aim)
print(gs.best_params_['C'], gs.best_score_, gs.grid_scores_)


print ('Time elapsed:', datetime.datetime.now() - start_time)

Features with NaN: Index(['first_blood_time', 'first_blood_team', 'first_blood_player1',
       'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time',
       'radiant_flying_courier_time', 'radiant_first_ward_time',
       'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time',
       'dire_first_ward_time'],
      dtype='object') 12
0.01 0.716341465365 [mean: 0.69512, std: 0.00263, params: {'C': 1.0000000000000001e-05}, mean: 0.71125, std: 0.00271, params: {'C': 0.0001}, mean: 0.71618, std: 0.00275, params: {'C': 0.001}, mean: 0.71634, std: 0.00281, params: {'C': 0.01}, mean: 0.71631, std: 0.00283, params: {'C': 0.10000000000000001}, mean: 0.71631, std: 0.00283, params: {'C': 1.0}, mean: 0.71631, std: 0.00283, params: {'C': 10.0}, mean: 0.71631, std: 0.00283, params: {'C': 100.0}, mean: 0.71631, std: 0.00283, params: {'C': 1000.0}, mean: 0.71631, std: 0.00283, params: {'C': 10000.0}, mean: 0.71631, std: 0.00283, params: {'C': 100000.0}]
Time elapsed: 0:03:2

In [13]:
#Задание 2. Удалим категорные признаки из выборки. Снова обучим Логистическую регрессию.
categorial_features = ['lobby_type'] + ['r' + str(i) + '_hero' for i in range(1,6)] + ['d' + str(i) + '_hero' for i in range(1,6)]
non_cat_feat = features.drop(categorial_features, axis = 1)
scaler_two = StandardScaler()
non_cat_feat_scaled = scaler_two.fit_transform(non_cat_feat)
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cr_v = KFold(aim.size, n_folds = 5, shuffle = True, random_state = 241)
clf = LogisticRegression(random_state = 241)
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv = cr_v)
gs.fit(non_cat_feat_scaled, aim)
print(gs.best_params_['C'], gs.best_score_, gs.grid_scores_)
#Качество немного улучшилось - в 4 знаке после запятой. Улучшилось - потому что категорные признаки, использованные как числовые,
#могут ослабить результат. С другой стороны, видимо, они имели небольшой вес, поэтому их удаление почти не повлияло на качество.

0.01 0.716400950653 [mean: 0.69506, std: 0.00265, params: {'C': 1.0000000000000001e-05}, mean: 0.71125, std: 0.00276, params: {'C': 0.0001}, mean: 0.71624, std: 0.00280, params: {'C': 0.001}, mean: 0.71640, std: 0.00285, params: {'C': 0.01}, mean: 0.71637, std: 0.00286, params: {'C': 0.10000000000000001}, mean: 0.71637, std: 0.00286, params: {'C': 1.0}, mean: 0.71637, std: 0.00286, params: {'C': 10.0}, mean: 0.71637, std: 0.00286, params: {'C': 100.0}, mean: 0.71637, std: 0.00286, params: {'C': 1000.0}, mean: 0.71637, std: 0.00286, params: {'C': 10000.0}, mean: 0.71637, std: 0.00286, params: {'C': 100000.0}]


In [38]:
#Задание 3. Выясним количество идентификаторов героев в игре.
heroes = [ind for ind in features.columns if 'hero' in ind]
unique_heroes = np.unique(features[heroes])
print(len(unique_heroes), unique_heroes) #То есть уникальных героев у нас 108, нескольких не хватает, т.к. максимальный номер 112.
#Но мешок слов в следующем задании будем составлять, используя приведенный в задании код,возьмем там N=112, чтобы он корректно работал

108 [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 109 110 112]


In [5]:
#Пришлось еще раз скопировать код с началом программы, потому что из-за memory error перезапустилось ядро Ipython
#Случайно еще раз напечатал признаки с пропусками
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

features = pd.read_csv('features.csv', index_col='match_id')

aim = features['radiant_win']

features = features.drop(['radiant_win', 'duration','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'], axis = 1)

features_with_nans = np.nonzero(features.count(axis = 0) < 97230)
print("Features with NaN:", features.axes[1][features_with_nans], len(features.axes[1][features_with_nans]))

features = features.fillna(value = 0)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
categorial_features = ['lobby_type'] + ['r' + str(i) + '_hero' for i in range(1,6)] + ['d' + str(i) + '_hero' for i in range(1,6)]
non_cat_feat = features.drop(categorial_features, axis = 1)
scaler_two = StandardScaler()
non_cat_feat_scaled = scaler_two.fit_transform(non_cat_feat)
#Задание 4. Составление мешка слов.
test = pd.read_csv('features_test.csv', index_col='match_id')
test = test.fillna(0)
non_cat_test = test.drop(categorial_features, axis = 1)
non_cat_test_scaled = scaler_two.transform(non_cat_test)

bag = np.zeros((features.shape[0], 112))
#Берем N = 112, хотя уникальных игроков 108. У нас могут появиться лишние пропуски, но нули на алгоритм не влияют. 
#Не забываем параллельно создавать мешок слов и для тестовой выборки

for i, match_id in enumerate(features.index):
    for p in range(5):
        bag[i, features.ix[match_id, 'r%d_hero' % (p+1)] - 1] = 1
        bag[i, features.ix[match_id, 'd%d_hero' % (p+1)] - 1] = -1
        
bag_test = np.zeros((test.shape[0], 112))
for i, match_id in enumerate(test.index):
    for p in range(5):
        bag_test[i, test.ix[match_id, 'r%d_hero' % (p+1)] - 1] = 1
        bag_test[i, test.ix[match_id, 'd%d_hero' % (p+1)] - 1] = -1
#Склеим теперь мешок слов и выборки с отмасштабированными весами. Мешок уже отмасштабирован, не будем его редактировать намеренно.
train_result = np.hstack((non_cat_feat_scaled, bag))
test_result = np.hstack((non_cat_test_scaled, bag_test))
#Задание 5. Обучаем регрессию на новой выборке с учетом мешка слов.
#Снова проводим обучение регрессии и подбор параметра С:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cr_v = KFold(aim.size, n_folds = 5, shuffle = True, random_state = 241)
clf = LogisticRegression(random_state = 241)
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv = cr_v)
gs.fit(train_result, aim)
print(gs.best_params_['C'], gs.best_score_)
#Видим, что качество существенно выросло после использования мешка слов по сравнению с прошлым результатом. Этого удалось достичь,
#так как мы извлекли информацию из категориальных признаков, которые до этого просто выбросили, информация о героях является
#очень важной, исходя из здравого смысла.

Features with NaN: Index(['first_blood_time', 'first_blood_team', 'first_blood_player1',
       'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time',
       'radiant_flying_courier_time', 'radiant_first_ward_time',
       'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time',
       'dire_first_ward_time'],
      dtype='object') 12
0.1 0.75193745059


In [7]:
#Задание 6. Теперь применим нашу модель к тестовой выборке, предскажем исходы матчей. Применять будем Логистическую регрессию с 
#мешком слов, так как она дала наилучший результат из всех вариантов.
clf = LogisticRegression(C = 0.1, random_state = 241)
clf.fit(train_result, aim)
prediction = clf.predict_proba(test_result)
print(np.amax(prediction), np.amin(prediction)) 
#Найдя наибольшее и наименьшее значение вероятностей, убеждаемся, что они не совпадают, то есть модель не обучилась константой.
#Кроме того, наименьшее и наибольшее значение лежат на [0,1], значит и все значения - тоже. Что и требовалось.

0.996371448815 0.00362855118457
